# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [87]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [52]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [53]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [54]:
# to get price from one api call, real time
symbol = 'IBM'
import creds
url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={creds.api_key}'
r = requests.get(url)
data = r.json()
# print(data)

# api call to get other data
overview_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={creds.api_key}'
req = requests.get(overview_url)
overview = req.json()
print(overview)

{'Symbol': 'IBM', 'AssetType': 'Common Stock', 'Name': 'International Business Machines', 'Description': 'International Business Machines Corporation (IBM) is an American multinational technology company headquartered in Armonk, New York, with operations in over 170 countries. The company began in 1911, founded in Endicott, New York, as the Computing-Tabulating-Recording Company (CTR) and was renamed International Business Machines in 1924. IBM is incorporated in New York. IBM produces and sells computer hardware, middleware and software, and provides hosting and consulting services in areas ranging from mainframe computers to nanotechnology. IBM is also a major research organization, holding the record for most annual U.S. patents generated by a business (as of 2020) for 28 consecutive years. Inventions by IBM include the automated teller machine (ATM), the floppy disk, the hard disk drive, the magnetic stripe card, the relational database, the SQL programming language, the UPC barcod

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [55]:
# to get price
print(data.keys())
price = data['Global Quote']['05. price']


# to get market cap
print(overview.keys())
market_cap = overview['MarketCapitalization']

print("price", price)
print("market cap ", market_cap)

dict_keys(['Global Quote'])
dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'OfficialSite', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'AnalystRatingStrongBuy', 'AnalystRatingBuy', 'AnalystRatingHold', 'AnalystRatingSell', 'AnalystRatingStrongSell', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])
price 224.4100
market cap  207499592000


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [56]:
my_columns = ['ticker', 'stock price', 'market capitalization', 'number of shares to buy']
final_data_frame = pd.DataFrame(columns = my_columns)
final_data_frame

,ticker,stock price,market capitalization,number of shares to buy


In [57]:
final_data_frame.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            "N/A"
        ],
        index = my_columns
    ), ignore_index = True
)

/var/folders/2p/819b3_bd519_rzy941d__cmm0000gn/T/ipykernel_1145/2235372101.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data_frame.append(


,ticker,stock price,market capitalization,number of shares to buy
0,IBM,224.4100,207499592000,N/A


In [58]:
#using Yahoo finance
import yfinance as yf
tick = yf.Ticker('GOOGL')
# print(tick.info)
current_price = tick.info["currentPrice"]

# Market cap
market_cap = tick.info["marketCap"]

# Shares outstanding
shares_outstanding = tick.info["sharesOutstanding"]

print(f"Price: {current_price}, Market Cap: {market_cap}, Shares Outstanding: {shares_outstanding}")




Price: 196.11, Market Cap: 2408662171648, Shares Outstanding: 5842999808


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
data_rows = []
final_data_frame = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
#     print(stock)
    try:
        ticker = yf.Ticker(stock)
        info = ticker.info
        stock_price = info.get("currentPrice")
        market_cap = info.get("marketCap")
        
        data_rows.append(
        {
            'ticker': stock,
            'stock price': stock_price,
            'market capitalization': market_cap,
            'number of shares to buy':'N/A'
        })
    except Exception as e:
        print(f"Error fetching data for {stock}: {e}")

final_data_frame = pd.concat([final_data_frame, pd.DataFrame(data_rows)], ignore_index=True)



In [60]:
final_data_frame

,ticker,stock price,market capitalization,number of shares to buy


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [59]:
def chunks(lst,n):
    "splits the list"
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [60]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_groups
symbol_strings = []

In [61]:
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

#clears the data frame
# final_data_frame = pd.DataFrame(columns = my_columns)
final_data = []
for symbol_string in symbol_strings:
    symbols = yf.Tickers(symbol_string) #implementing batch call, this is single API call
    sym_dict = symbols.tickers #ticker dict
    
    for ticker, ticker_obj in sym_dict.items():  # Loop through the dictionary
        try:
            info = ticker_obj.info  # Get the 'info' dictionary
            row = {
                'ticker': ticker,
                'stock price': info.get('currentPrice', 'N/A'),
                'market capitalization': info.get('marketCap', 'N/A'),
                'number of shares to buy': "N/A"
            }
            final_data.append(row)  # Append to the list
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
final_data_frame = pd.DataFrame(final_data, columns=my_columns)


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=G2wdu.afGIe
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=G2wdu.afGIe
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=G2wdu.afGIe
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.

In [62]:
final_data_frame

,ticker,stock price,market capitalization,number of shares to buy
0,A,136.1,38869483520,N/A
1,AAL,17.35,11401223168,N/A
2,AAP,43.73,2612189696,N/A
3,AAPL,258.2,3902899748864,N/A
4,ABBV,180.0,318085201920,N/A
...,...,...,...,...
500,YUM,135.0,37674721280,N/A
501,ZBH,107.3,21360640000,N/A
502,ZBRA,395.44,20396795904,N/A
503,ZION,54.97,8119674368,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [79]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('please enter an integer')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the value of your portfolio: 10000000
10000000.0


In [80]:
position_size = val / len(final_data_frame.index)
position_size

19801.980198019803

In [81]:

#calculate how many shares of each stock has to be purchased, ex appl
# num_of_apple_shares = math.floor(position_size / 500) #we need to round down
# num_of_apple_shares


for i in range(0,len(final_data_frame.index)):
    # way to access specific data using pandas .loc[row, column name]
    sp = final_data_frame.loc[i,'stock price']
    if type(sp) != str:
        final_data_frame.loc[i, 'number of shares to buy'] = math.floor(position_size / sp)
    else:
        final_data_frame.loc[i, 'number of shares to buy'] = 'N/A'

final_data_frame
    
    

,ticker,stock price,market capitalization,number of shares to buy
0,A,136.1,38869483520,145
1,AAL,17.35,11401223168,1141
2,AAP,43.73,2612189696,452
3,AAPL,258.2,3902899748864,76
4,ABBV,180.0,318085201920,110
...,...,...,...,...
500,YUM,135.0,37674721280,146
501,ZBH,107.3,21360640000,184
502,ZBRA,395.44,20396795904,50
503,ZION,54.97,8119674368,360


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [110]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_data_frame.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [111]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }

)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }

)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }

)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [102]:
# writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
# writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [114]:
column_formats = {
    'A':['ticker', string_format],
    'B': ['stock price', dollar_format],
    'C': ['market capitalization', dollar_format ],
    'D': ['number of shares to buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)


## Saving Our Excel Output

Saving our Excel file is very easy:

In [115]:
writer.save()